In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Computing eigendecomposition using `np.linalg.eigh`

In [145]:
n = 3
M = np.random.uniform(-1, 1, (n, n))
M = 0.5 * M + 0.5 * M.T

In [146]:
M

array([[ 0.95818468, -0.41153984, -0.12721255],
       [-0.41153984, -0.577583  ,  0.3039353 ],
       [-0.12721255,  0.3039353 , -0.41524987]])

In [147]:
evals, evecs = np.linalg.eigh(M)

In [148]:
evals

array([-0.84919519, -0.27358113,  1.08812814])

In [149]:
evecs

array([[-0.15118287,  0.25081626, -0.95615634],
       [-0.82994262,  0.49323197,  0.26060979],
       [ 0.53697205,  0.83295464,  0.13359488]])

In [150]:
for i in range(n):
    x = evecs[:, i]
    diff = M.dot(x) - evals[i] * x
    print(np.linalg.norm(diff))

1.49468349007e-16
1.20983749228e-16
3.18887285829e-16


## Computing eigendecomposition using symbolic algebra (for small matrices only)

First, compute the characteristic polynomial $\det(A - \lambda I) = 0$

In [92]:
import sympy as sp

In [93]:
lam = sp.symbols("lam")

In [134]:
A = np.array(sp.symbols('a:{}:{}'.format(n, n))).reshape((n,n))
L = np.diag([lam for i in range(n)])

In [135]:
def det(A):
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    if n == 0:
        return None
    elif n == 1:
        return A[0,0]
    elif n == 2:
        return A[0,0] * A[1,1] - A[1,0] * A[0,1]
    else:
        result = 0
        for i in range(n):
            sign = 1 if i % 2 == 0 else -1
            submatrix = np.hstack([A[1:, :i], A[1:, i+1:]])
            result += sign * A[0,i] * det(submatrix)
        return result

In [136]:
det(A)

a00*(a11*a22 - a12*a21) - a01*(a10*a22 - a12*a20) + a02*(a10*a21 - a11*a20)

In [137]:
det(A - L)

-a01*(a10*(a22 - lam) - a12*a20) + a02*(a10*a21 - a20*(a11 - lam)) + (a00 - lam)*(-a12*a21 + (a11 - lam)*(a22 - lam))

In [152]:
sol = sp.solve(sp.Eq(det(A - L), 0), lam)

In [153]:
len(sol)

3

In [154]:
x = sp.symbols('x:{}'.format(n))

In [155]:
list(np.dot(A - L, x))

[a01*x1 + a02*x2 + x0*(a00 - lam),
 a10*x0 + a12*x2 + x1*(a11 - lam),
 a20*x0 + a21*x1 + x2*(a22 - lam)]

In [158]:
%%time
subs = dict()
for i in range(n):
    for j in range(n):
        subs['a{}{}'.format(i,j)] = M[i, j]

sym_evals = [sol[i].evalf(n = 5, subs = subs, chop = True) for i in range(n)]
print(sym_evals)

[-0.84920, -0.27358, 1.0881]
CPU times: user 22.8 s, sys: 174 ms, total: 23 s
Wall time: 23.3 s


In [160]:
eqns = [sp.Eq(row, 0) for row in np.dot(A - L, x)]
print(eqns)

sol_ev = sp.solve(eqns[:-1], *x[:-1])
print(sol_ev)

ev = [sol_ev[x[i]] for i in range(n-1)] + [x[-1]]
print(ev)

[Eq(a01*x1 + a02*x2 + x0*(a00 - lam), 0), Eq(a10*x0 + a12*x2 + x1*(a11 - lam), 0), Eq(a20*x0 + a21*x1 + x2*(a22 - lam), 0)]
{x0: x2*(-a01*a12 + a02*(a11 - lam))/(a01*a10 - (a00 - lam)*(a11 - lam)), x1: x2*(-a02*a10 + a12*(a00 - lam))/(a01*a10 - (a00 - lam)*(a11 - lam))}
[x2*(-a01*a12 + a02*(a11 - lam))/(a01*a10 - (a00 - lam)*(a11 - lam)), x2*(-a02*a10 + a12*(a00 - lam))/(a01*a10 - (a00 - lam)*(a11 - lam)), x2]


In [161]:
sym_evecs = []
for i in range(n):
    current_subs = dict()
    current_subs.update(subs)
    current_subs[x[-1]] = 1
    current_subs[lam] = sym_evals[i]
    
    v = [float(evi.evalf(n = 5, subs = current_subs, chop = True)) for evi in ev]
    v /= np.linalg.norm(v)
    sym_evecs.append(v)

sym_evecs = np.asarray(sym_evecs).transpose()
sym_evecs

array([[-0.15118281,  0.25081641, -0.95615714],
       [-0.82994227,  0.49323159,  0.26060935],
       [ 0.53697261,  0.83295482,  0.13358997]])

## Computing eigendecomposition via power iteration

Select a random initial vector $x_0$.
Iteratively compute $ x_{k+1} = M x_k $ and $ \mu_k = x_k^T x_{k+1} / \lVert x_k \rVert $
until $\mu_k$ approximately converges.

As $k \to \infty$, we will have $\mu_k \to \lambda_1$ (the largest-magnitude eigenvalue) and $x_k \to v_1$ (the corresponding eigenvector).

"Deflate" the matrix via $M \gets M - \lambda_1 v_1 v_1^T$ and repeat.

In [293]:
def power_iteration(M, max_iterations = int(1e5), eps = 1e-10):
    n = M.shape[0]
    x = np.random.uniform(-1, 1, (n,))
    x /= np.linalg.norm(x)
    prev_estimate = None

    for count in range(max_iterations):
        x_old = x
        Mx = M.dot(x)
        x = Mx / np.linalg.norm(Mx)

        eigval_estimate = x_old.transpose().dot(Mx) / np.linalg.norm(x_old)
        if prev_estimate is not None and abs(prev_estimate - eigval_estimate) < eps:
            break
        prev_estimate = eigval_estimate

    eigvec_estimate = x

    if count >= max_iterations - 1:
        print "WARNING: high number of iterations"
    
    return eigval_estimate, eigvec_estimate

In [324]:
def multiple_power_iteration(X):
    ms, vs = [], []
    for i in range(X.shape[0]):
        m, v = power_iteration(X)
        ms.append(m)
        vs.append(v)
        X = X - m * np.outer(v, v)
    ms = np.array(ms)
    vs = np.array(vs).transpose()
    indices = np.argsort(ms)
    ms = ms[indices]
    vs = vs[:, indices]
    return ms, vs

In [332]:
ms, vs = multiple_power_iteration(M)

for i in range(n):
    print((ms[i], vs[:, i]))

(-0.849195191060649, array([-0.15119011, -0.82993996,  0.53697413]))
(-0.27358113395603945, array([-0.25081683, -0.49323515, -0.83295258]))
(1.0881281350063867, array([-0.95615727,  0.26060468,  0.13359819]))


In [333]:
assert np.allclose(evals, ms, atol=1e-5)

In [334]:
for i in range(n):
    print M.dot(vs[:, i]) - ms[i] * vs[:, i]

[ -1.44483700e-05   4.33673438e-06   2.63511017e-06]
[  3.33169407e-07   1.82921616e-06  -1.18349171e-06]
[  1.80473037e-06   9.90802250e-06  -6.41044414e-06]
